# Langchain into

<img src="images/langchain_overview.png" width=800px />

## Компоненты LangChain

- **langchain-core**  
  Базовые абстракции для ключевых компонентов (модели, хранилища векторов, инструменты) и их компоновки. Легковесный, без сторонних интеграций.

- **langchain**  
  Общие цепочки и стратегии извлечения данных для построения когнитивной архитектуры приложений. Не привязан к сторонним системам.

- **Integration Packages**  
  Отдельные пакеты для популярных интеграций (например, `langchain-openai`, `langchain-anthropic`), чтобы обеспечить гибкость версий и минимальные зависимости.

- **langchain-community**  
  Сторонние интеграции, поддерживаемые сообществом. Включает интеграции для моделей, хранилищ и инструментов. Зависимости по умолчанию отключены.

- **langgraph**  
  Расширение для создания устойчивых, многоактных приложений с использованием графовой модели шагов.

- **langserve**  
  Инструмент для развертывания цепочек LangChain в виде REST API, подходящий для простых сценариев и известных примитивов.

- **LangSmith**  
  Платформа для разработчиков для отладки, тестирования, оценки и мониторинга LLM-приложений.

## 1. Langchain models


In [ ]:
# pip install langchain
# pip install langchain-openai

In [19]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from pprint import pp

# Load environment variables from .env
load_dotenv(override=True)

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o-mini", verbose=True)

# Invoke the model with a message
result = model.invoke("Кто тебя создал?")
print("Full result:")
print(result)
print("Content only:")
print(result.content)

PermissionDeniedError: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}

In [3]:
from langchain.globals import set_verbose
from langchain.globals import set_debug
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

set_verbose(False)
set_debug(False)

messages = [
    SystemMessage(content="Ты - ИИ помощник, ты помогаешь людям."),
    HumanMessage(content="Давай познакомимся. Меня зовут Евгений, а тебя?"),
]


# Invoke the model with messages
ai_answer = model.invoke(messages)
print(type(ai_answer))
print(f"Answer from AI: {ai_answer.content}")


# AIMessage:
#   Message from an AI.
messages.append(ai_answer)
messages.append(HumanMessage(content="Напомни мне, как меня зовут?"))

# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")


<class 'langchain_core.messages.ai.AIMessage'>
Answer from AI: Приятно познакомиться, Евгений! Я — твой виртуальный помощник, и у меня нет имени, как у человека, но я здесь, чтобы помочь тебе. Чем могу помочь?
Answer from AI: Конечно, ты сказал, что тебя зовут Евгений. Если я могу чем-то помочь, дай знать!


In [ ]:
# pip install grpcio
# pip install yandexcloud
# pip install langchain-community


In [ ]:
from langchain_together import ChatTogether
from yandex_cloud_ml_sdk import YCloudML
from langchain_core.messages import AIMessage, HumanMessage


model = ChatTogether(model='Qwen/Qwen2.5-Coder-32B-Instruct', max_tokens=1024)
sdk = YCloudML(folder_id=os.environ.get('YC_FOLDER_ID'), auth=os.environ.get('YC_API_KEY'))
model_yandex = sdk.models.completions('yandexgpt').langchain(model_type="chat", timeout=60)

langchain_result = model.invoke([
    HumanMessage(content="hello!"),
    AIMessage(content="Hi there human!"),
    HumanMessage(content="Meow!"),
])
messages = [
    SystemMessage(content="Ты - ИИ помощник, ты помогаешь людям."),
    HumanMessage(content="Давай познакомимся. Меня зовут Евгений, а тебя?"),
]

# Invoke the model with messages
ai_answer = model.invoke(messages)
print(f"Answer from AI: {ai_answer}")
print(type(ai_answer))

# AIMessage: Message from an AI.
messages.append(ai_answer)
messages.append(HumanMessage(content="Напомни мне, как меня зовут?"))

# # Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

Answer from AI: content='Привет, Евгений! Рад познакомиться. Я — ИИ-помощник, созданный для того, чтобы помогать людям. Чем могу тебе помочь сегодня?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 53, 'total_tokens': 103, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen2.5-Coder-32B-Instruct', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None} id='run-499e79a6-384e-4d0f-a614-22414e88af7d-0' usage_metadata={'input_tokens': 53, 'output_tokens': 50, 'total_tokens': 103, 'input_token_details': {}, 'output_token_details': {}}
<class 'langchain_core.messages.ai.AIMessage'>
Answer from AI: content='Вы сказали мне, что вас зовут Евгений. Если вам нужно что-то еще, не стесняйтесь спрашивать!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 124, 'total_tokens': 159, 'completion_tokens_details': None, 'pro

In [22]:
pp(messages)
print(10*"==")
pp([message.content for message in messages])
# # Invoke the model with messages YandexGPT
result = model_yandex.invoke(messages)
print(f"Answer from AI: {result.content}")


[SystemMessage(content='Ты - ИИ помощник, ты помогаешь людям.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Давай познакомимся. Меня зовут Евгений, а тебя?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Привет, Евгений! Рад познакомиться. Я — ИИ-помощник, созданный для того, чтобы помогать людям. Чем могу тебе помочь сегодня?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 53, 'total_tokens': 103, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen2.5-Coder-32B-Instruct', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-499e79a6-384e-4d0f-a614-22414e88af7d-0', usage_metadata={'input_tokens': 53, 'output_tokens': 50, 'total_tokens': 103, 'input_token_details': {}, 'output_token_details': {}}),
 HumanMessage(content='Напомни мне, как меня зовут?', additional_kwargs={}, response_metadata={})]
['Ты - ИИ помощни

## 2. Prompts templates

In [ ]:
from langchain.prompts import ChatPromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


# # PART 1: Create a ChatPromptTemplate using a template string
template = "Придумай короткую шутку про {topic}."
prompt_template = ChatPromptTemplate.from_template(template)

print("-----Prompt from Template-----")
prompt = prompt_template.invoke({"topic": "крокодил"})
print(prompt)

# # PART 2: Prompt with Multiple Placeholders
template_multiple = """Ты - ИИ-помощник, ты помогаешь людям.
Human: Расскажи мне короткую {adjective} историю о {subject}."""
prompt_multiple = ChatPromptTemplate.from_template(template_multiple)
prompt = prompt_multiple.invoke({"adjective": "веселую", "subject": "крокодиле"})
print("\n----- Prompt with Multiple Placeholders -----\n")
print(prompt)


# PART 3: Prompt with System and Human Messages (Using Tuples)
messages = [
    ("system", "Ты выступаешь на стендапе и рассказываешь шутки на тему {topic}."),
    ("human", "Расскажи мне {joke_count} шуток."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "ИТ", "joke_count": 3})
print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
print(prompt)

# # Extra Informoation about Part 4.
# # This does work:
# messages = [
#     ("system", "You are a comedian who tells jokes about {topic}."),
#     HumanMessage(content="Tell me 3 jokes."),
# ]
# prompt_template = ChatPromptTemplate.from_messages(messages)
# prompt = prompt_template.invoke({"topic": "lawyers"})
# print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
# print(prompt)

## PART 4: Prompt with System and Human Messages templates
messages = [
SystemMessagePromptTemplate.from_template('Ты выступаешь на стендапе и рассказываешь шутки на тему {topic}.'),
HumanMessagePromptTemplate.from_template('Расскажи мне {joke_count} шуток.')
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "ИТ", "joke_count": 3})
print("\n----- Prompt with System and Human Messages templates -----\n")
print(prompt)


-----Prompt from Template-----
messages=[HumanMessage(content='Придумай короткую шутку про крокодил.', additional_kwargs={}, response_metadata={})]

----- Prompt with Multiple Placeholders -----

messages=[HumanMessage(content='Ты - ИИ-помощник, ты помогаешь людям.\nHuman: Расскажи мне короткую веселую историю о крокодиле.', additional_kwargs={}, response_metadata={})]

----- Prompt with System and Human Messages (Tuple) -----

messages=[SystemMessage(content='Ты выступаешь на стендапе и рассказываешь шутки на тему ИТ.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Расскажи мне 3 шуток.', additional_kwargs={}, response_metadata={})]

----- Prompt with System and Human Messages templates -----

messages=[SystemMessage(content='Ты выступаешь на стендапе и рассказываешь шутки на тему ИТ.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Расскажи мне 3 шуток.', additional_kwargs={}, response_metadata={})]


## 3. LCEL - LangChainExpression Language



In [23]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = model_yandex

# Define prompt templates (no need for separate Runnable chains)
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты придумываешь шутки на тему {topic}."),
        ("human", "Напиши {joke_count} коротких смешных шутки."),
    ]
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser()
# chain = prompt_template | model

# Run the chain
result = chain.invoke({"topic": "ИТ-разпработчики", "joke_count": 3})

# Output
print(result)

1. Два программиста сидят в кафе и обсуждают свои проекты. Один говорит:
— Я разрабатываю приложение, которое поможет людям находить потерянные вещи.
Второй отвечает:
— А я создаю программу, которая будет предсказывать будущее.
Первый улыбается:
— Тогда давай встретимся здесь через год и посмотрим, кто из нас окажется ближе к цели!

2. Программист приходит на собеседование в компанию и видит, что все сотрудники выглядят очень счастливыми. Он спрашивает у HR-менеджера:
— Почему все так довольны?
HR отвечает:
— Мы используем нашу собственную систему мотивации, основанную на алгоритмах машинного обучения. Она анализирует работу каждого сотрудника и выдаёт им бонусы за хорошую работу.
Программист улыбается:
— И как, работает?
HR кивает:
— Ещё как! Все сотрудники теперь работают в два раза больше, чтобы получить больше бонусов!

3. Два программиста встречаются в коридоре офиса и начинают обсуждать свои последние проекты. Первый говорит:
— У меня есть идея для нового приложения, которое буде

In [25]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableSequence
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
# model = ChatOpenAI(model="gpt-4")
model = model_yandex

# Define prompt templates
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты придумываешь шутки на тему {topic}."),
        ("human", "Напиши {joke_count} коротких смешных шутки."),
    ]
)

# Create individual runnables (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x)

# Create the RunnableSequence (equivalent to the LCEL chain)
chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)

# Run the chain
response = chain.invoke({"topic": "сисадмины", "joke_count": 3})

# Output
print(response.content)

1. Знаете, как сисадмин понимает, что сегодня выходной? У него в календаре всего два рабочих созвона!

2. Сисадмин настолько суров, что для него даже «синий экран смерти» — это просто приглашение к диалогу.

3. Почему сисадмины не любят рассказывать о своих достижениях? Потому что они предпочитают говорить только о проблемах, которые решили.


In [26]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
# model = ChatOpenAI(model="gpt-4o")
model = model_yandex

# Define prompt templates
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты - резидент ComedyClub. Ты шутишь на тему {topic}."),
        ("human", "Расскажи мне {joke_count} шутки."),
    ]
)

# Define additional processing steps using RunnableLambda
uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser() | uppercase_output | count_words

# Run the chain
result = chain.invoke({"topic": "Программисты", "joke_count": 3})

# Output
print(result)

Word count: 41
1. ЗНАЕТЕ, ПОЧЕМУ ПРОГРАММИСТЫ НЕ ЛЮБЯТ ПРИРОДУ? ПОТОМУ ЧТО У НИХ И ТАК СВОЙ «КОД ДА ВИНЧИ»!

2. ПРОГРАММИСТЫ НАСТОЛЬКО СУРОВЫ, ЧТО ВМЕСТО «ПРИВЕТ!» ОНИ ОБМЕНИВАЮТСЯ «ХЕЛЛОУВИНГАМИ».

3. ПОЧЕМУ ПРОГРАММИСТ НЕ МОЖЕТ БЫТЬ РОМАНТИКОМ? ПОТОМУ ЧТО ОН ВСЕГДА НА «КОНЕЧНОЙ» СТАНЦИИ!


In [27]:
# parallel chains

from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
# model = ChatOpenAI(model="gpt-4o")
model = model_yandex

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert an cats and dogs. Answer in Russian."),
        ("human", "List the main features of the animal type {product_name}."),
    ]
)


# Define pros analysis step
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert an cats and dogs"),
            (
                "human",
                "Given these features: {features}, list the 3 pros of these animal.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)


# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert an cats and dogs"),
            (
                "human",
                "Given these features: {features}, list the 3 cons of these animal.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)


# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"


# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

# Run the chain
result = chain.invoke({"product_name": "Кошка породы Корниш рекс"})

# Output
print(result)

Pros:
1. Корниш-рексы обладают ласковым и дружелюбным характером, они очень привязаны к своим хозяевам и любят внимание.
2. Эти кошки умны и любознательны, их легко обучать различным трюкам.
3. У корниш-рексов уникальная шерсть, которая вьётся или волнится и плотно прилегает к телу. Они имеют большие уши и выразительные глаза, а также стройное и изящное телосложение.

Cons:
1. Корниш-рексы нуждаются в регулярном купании, так как их кожа выделяет специальный секрет. Это может быть неудобно для некоторых владельцев.

2. Хотя корниш-рексы — это здоровая порода, они могут быть склонны к некоторым генетическим заболеваниям. Поэтому важно тщательно выбирать котёнка и регулярно проходить ветеринарные осмотры.
3. Как и любой другой кошке, корниш-рексу требуется сбалансированное питание, богатое белками, витаминами и минералами. Неправильный рацион может привести к проблемам со здоровьем.


In [30]:
# branches chains

from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
# model = ChatOpenAI(model="gpt-4o")

model = model_yandex

system_prompt = "You are a helpful assistant. Answer always in Russian."
# Define prompt templates for different feedback types
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human",
         "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human",
         "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | model | StrOutputParser()
)

# Create the classification chain
classification_chain = classification_template | model | StrOutputParser()

# Combine classification and response generation into one chain
chain = classification_chain | branches

# Run the chain with an example review
# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

review = "The product is terrable"
result = chain.invoke({"feedback": review})

# Output the result
print(result)

Здравствуйте!

Прошу обратить внимание на отзыв пользователя. Клиент остался недоволен. Прошу связаться с ним для решения проблемы.


# 3. LangSmith

# 4. LangGraph